In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [7]:
print("Archivos de estudiantes: ", estuDic['20161'],estuDic['20162'])
print("Archivos de colegios: ",coleDic['20161'],coleDic['20162'])

Archivos de estudiantes:  SB11_20161.txt SB11_20162.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-20161.txt SB11-CLASIFI-PLANTELES-20162.txt


In [8]:
df1 = spark.read.load(base2 + '/' + estuDic['20161'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [9]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_TIENEETNIA',
 'ESTU_ETNIA',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_NUMHERMANOS',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENESERVICIOTV',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TELEFONO',
 'FAMI_NUMLIBROS',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'COLE_CODIGO_ICFES',
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'COLE_GENERO

In [10]:
columnasPuntos20161 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_INGLES']

In [11]:
df1 = df1.select(columnasPuntos20161)

In [12]:
# SE leen los dos archivos y se crean dos dataframes
df2 = spark.read.load(base2 + '/' + estuDic['20162'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [13]:
df2.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_TIENEETNIA',
 'ESTU_ETNIA',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_NUMHERMANOS',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENESERVICIOTV',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TELEFONO',
 'FAMI_NUMLIBROS',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'COLE_CODIGO_ICFES',
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'COLE_GENERO

In [14]:
columnasPuntos20162 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_INGLES']

In [15]:
df2 = df2.select(columnasPuntos20162)

In [16]:
#dft = df1.columns.remove('cole_cod_dane_institucion')
cols = df1.columns
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols.remove('COLE_NOMBRE_ESTABLECIMIENTO')
cols

['PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_INGLES']

In [17]:
for cp in cols:
    df1 = df1.withColumn(cp, regexp_replace(cp, ',', '.'))
    df1 = df1.withColumn(cp, df1[cp].cast("float"))
    df1 = df1.na.fill({cp:0})

In [18]:
df1 = df1.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df1['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [19]:
cols2 = df2.columns
cols2.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols2.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [20]:
for cp in cols2:
    df2 = df2.withColumn(cp, regexp_replace(cp, ',', '.'))
    df2 = df2.withColumn(cp, df2[cp].cast("float"))
    df2 = df2.na.fill({cp:0})
df2 = df2.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df2['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [21]:
print("Numero de filas primero semestre ", df1.count())
print("Numero de filas segundo semestre ", df2.count())

Numero de filas primero semestre  13065
Numero de filas segundo semestre  548206


In [22]:
print(df1.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_LECTURA_CRITICA: float (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_C_NATURALES: float (nullable = false)
 |-- PUNT_SOCIALES_CIUDADANAS: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)

None


In [23]:
print(df2.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_LECTURA_CRITICA: float (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_C_NATURALES: float (nullable = false)
 |-- PUNT_SOCIALES_CIUDADANAS: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)

None


In [24]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)
dsCountEst2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [25]:
dsCountEst1.show(10,False)
# Numero de estudiantes por instituciòn primer semestre

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|376001028219                 |175  |
|376001026925                 |170  |
|376001000721                 |157  |
|376001001116                 |154  |
|311769003474                 |140  |
|311001010820                 |136  |
|376001007670                 |135  |
|311848001011                 |131  |
|376001000900                 |130  |
|376001000276                 |127  |
+-----------------------------+-----+
only showing top 10 rows



In [26]:
dsCountEst1.count()

347

In [27]:
dsCountEst2.show(10,False)
# Numero de estudiantes por instituciòn segundo semestre

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|105001000108                 |968  |
|105001013340                 |764  |
|111001024732                 |633  |
|176001005813                 |618  |
|305001003963                 |554  |
|105154000301                 |553  |
|313001006701                 |545  |
|105615000236                 |536  |
|311001105863                 |531  |
|305001005265                 |507  |
+-----------------------------+-----+
only showing top 10 rows



In [28]:
#Se crea una columna con el total de los puntajes 
df1= df1.withColumn("Suma",df1['PUNT_LECTURA_CRITICA']+df1['PUNT_MATEMATICAS']+df1['PUNT_C_NATURALES']+
                    df1['PUNT_SOCIALES_CIUDADANAS']+df1['PUNT_INGLES'])

df2= df2.withColumn("Suma",df2['PUNT_LECTURA_CRITICA']+df2['PUNT_MATEMATICAS']+df2['PUNT_C_NATURALES']+
                    df2['PUNT_SOCIALES_CIUDADANAS']+df2['PUNT_INGLES'])

## Resultados finales por instituciòn 1er semestre

In [29]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()
df1 = df1.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df1 = df1.orderBy('rank',ascending= True)

In [30]:
df1.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+-----------------------------+---------------------------+------------------+----+
|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO|avg(Suma)         |rank|
+-----------------------------+---------------------------+------------------+----+
|376001013441                 |COLEGIO BILINGUE DIANA OESE|427.0             |1   |
|305001015503                 |COLEGIO LEONARDO DA VINCI  |410.44444444444446|2   |
|311769003342                 |COL LOS NOGALES            |407.10714285714283|3   |
|368001000516                 |COL NUEVO CAMBRIDGE        |405.22033898305085|4   |
|311769000165                 |COL SAN JORGE DE INGLATERRA|405.0121951219512 |5   |
|311848002424                 |COL SAN CARLOS             |404.59615384615387|6   |
|311848000278                 |GIMN VERMONT               |404.4065934065934 |7   |
|368276000826                 |COL LA QUINTA DEL PUENTE   |404.25            |8   |
|466001005641                 |LIC CAMPESTRE DE PEREIRA   |403.6            

## Resultados finales por instituciòn 2do semestre

In [31]:
df2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()
df2 = df2.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df2 = df2.orderBy('rank',ascending= True)

In [32]:
df2.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+-----------------------------+------------------------------------------+------------------+----+
|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO               |avg(Suma)         |rank|
+-----------------------------+------------------------------------------+------------------+----+
|311001065489                 |LIC CAMPO DAVID                           |388.1666666666667 |1   |
|311001076634                 |COL NUEVO COLOMBO AMERICANO               |385.6842105263158 |2   |
|311001031088                 |COL NUEVA INGLATERRA                      |382.68518518518516|3   |
|308001001871                 |NUEVO COLEGIO DEL PRADO                   |382.12820512820514|4   |
|311001086320                 |COLOMBO BRITANICO - BILINGÜE INTERNACIONAL|381.0263157894737 |5   |
|311848000308                 |COL ANGLO AMERICANO                       |381.00847457627117|6   |
|311001005176                 |LIC NAVARRA                               |379.8048780487805 |7   |
|425175032

## Colegios primer semestre

In [33]:
# SE crea Dataframe de colegios primer semestre
dfc = spark.read.load(base + '/' + coleDic['20161'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc = dfc.withColumn('COLE_COD_DANE', dfc['COLE_COD_DANE'].cast("string"))
dfc = dfc.withColumn("rankIndiceTotal", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfc = dfc.orderBy('rankIndiceTotal',ascending= True)

In [34]:
print(dfc.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_COD_DEPTO: integer (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GRADO: integer (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- MATRICULADOS_ULTIMOS_3: integer (nullable = true)
 |-- EVALUADOS_ULTIMOS_3: integer (nullable = true)
 |-- INDICE_MATEMATICAS: string (nullable = true)
 |-- INDICE_C_NATURALES: string (nullable = true)
 |-- INDICE_SOCIALES_CIUDADANAS: string (nullable = true)
 |-- INDICE_LECTURA_CRITICA: string (nullable = true)
 |-- INDICE_INGLES: string (nullable = true)
 |-- INDICE_TOTAL: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- rankI

In [35]:
colsColegio = dfc.columns

In [41]:
dfc.select('rankIndiceTotal','COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL').show(10)

+---------------+-------------+--------------------+------------+
|rankIndiceTotal|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|
+---------------+-------------+--------------------+------------+
|              1| 376001013441|COLEGIO BILINGUE ...|       0,917|
|              2| 311769003342|     COL LOS NOGALES|      0,9152|
|              3| 311848002424|      COL SAN CARLOS|      0,9109|
|              4| 311769000165|COL SAN JORGE DE ...|      0,9065|
|              5| 311848000278|        GIMN VERMONT|      0,9028|
|              5| 411848002780|COL CAMPOALEGRE LTDA|      0,9028|
|              6| 311848002351|COL SANTA FRANCIS...|      0,9017|
|              7| 368276000826|COL LA QUINTA DEL...|      0,9008|
|              8| 368001000516| COL NUEVO CAMBRIDGE|       0,899|
|              9| 311769004292|COL SAN MATEO APO...|      0,8968|
+---------------+-------------+--------------------+------------+
only showing top 10 rows



In [42]:
print("El numero de colegios es: ", dfc.count())

El numero de colegios es:  317


## Colegios segundo semestre

In [43]:
# SE crea Dataframe de colegios primer semestre
dfc2 = spark.read.load(base + '/' + coleDic['20162'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))
dfc2 = dfc2.withColumn("rankIndiceTotal", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfc2 = dfc2.orderBy('rankIndiceTotal',ascending= True)

In [44]:
print(dfc2.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_COD_DEPTO: integer (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GRADO: integer (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- MATRICULADOS_ULTIMOS_3: integer (nullable = true)
 |-- EVALUADOS_ULTIMOS_3: integer (nullable = true)
 |-- INDICE_MATEMATICAS: string (nullable = true)
 |-- INDICE_C_NATURALES: string (nullable = true)
 |-- INDICE_SOCIALES_CIUDADANAS: string (nullable = true)
 |-- INDICE_LECTURA_CRITICA: string (nullable = true)
 |-- INDICE_INGLES: string (nullable = true)
 |-- INDICE_TOTAL: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- rankI

In [46]:
dfc2.select('rankIndiceTotal','COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL').orderBy('INDICE_TOTAL',ascending= False).show(10)

+---------------+-------------+--------------------+------------+
|rankIndiceTotal|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|
+---------------+-------------+--------------------+------------+
|              1| 311001065489|     LIC CAMPO DAVID|      0,9113|
|              2| 311001089221| INST ALBERTO MERANI|      0,9108|
|              3| 311001076634|COL NUEVO COLOMBO...|       0,908|
|              4| 311001005176|         LIC NAVARRA|       0,907|
|              5| 354518001139|CAMBRIDGE SCHOOL ...|      0,9048|
|              6| 311848000308| COL ANGLO AMERICANO|       0,903|
|              7| 311001086320|COLOMBO BRITANICO...|      0,9005|
|              8| 368001000729|COL  SAN PEDRO CL...|      0,8946|
|              9| 315001001842| COLEGIO LOS �NGELES|      0,8942|
|             10| 425214000149|          COL REFOUS|      0,8936|
+---------------+-------------+--------------------+------------+
only showing top 10 rows



In [47]:
print("El numero de colegios segundo semestre: ", dfc2.count())

El numero de colegios segundo semestre:  8872


## Anàlisis comparativo entre estudiantes y colegios primer semestre

In [48]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_COD_DANE)

In [49]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 278


In [50]:
dfjoinPorNom = df1.join(dfc, df1.COLE_NOMBRE_ESTABLECIMIENTO == dfc.COLE_INST_NOMBRE)

In [51]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoinPorNom.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 240


In [52]:
# Se cruza la informaciòn por el nombre del colegio - se evidencia que el còdigo DANE no coincide
dfjoin.select('rank','rankIndiceTotal','COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','INDICE_TOTAL').show(10)

+----+---------------+-----------------------------+---------------------------+------------+
|rank|rankIndiceTotal|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO|INDICE_TOTAL|
+----+---------------+-----------------------------+---------------------------+------------+
|   1|              1|                 376001013441|       COLEGIO BILINGUE ...|       0,917|
|   2|             11|                 305001015503|       COLEGIO LEONARDO ...|      0,8949|
|   3|              2|                 311769003342|            COL LOS NOGALES|      0,9152|
|   4|              8|                 368001000516|        COL NUEVO CAMBRIDGE|       0,899|
|   5|              4|                 311769000165|       COL SAN JORGE DE ...|      0,9065|
|   6|              3|                 311848002424|             COL SAN CARLOS|      0,9109|
|   7|              5|                 311848000278|               GIMN VERMONT|      0,9028|
|   8|              7|                 368276000826|       C

## Anàlisis comparativo entre estudiantes y colegios segundo semestre

In [53]:
dfjoin2 = df2.join(dfc2, df2.COLE_COD_DANE_ESTABLECIMIENTO == dfc2.COLE_COD_DANE)

In [54]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios:", dfjoin2.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios: 8849


In [55]:
dfjoin2.columns

['COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'avg(PUNT_LECTURA_CRITICA)',
 'avg(PUNT_MATEMATICAS)',
 'avg(PUNT_C_NATURALES)',
 'avg(PUNT_SOCIALES_CIUDADANAS)',
 'avg(PUNT_INGLES)',
 'avg(Suma)',
 'rank',
 'PERIODO',
 'COLE_COD_DANE',
 'COLE_INST_NOMBRE',
 'COLE_CODMPIO_COLEGIO',
 'COLE_MPIO_MUNICIPIO',
 'COLE_COD_DEPTO',
 'COLE_DEPTO_COLEGIO',
 'COLE_NATURALEZA',
 'COLE_GRADO',
 'COLE_CALENDARIO_COLEGIO',
 'COLE_GENEROPOBLACION',
 'MATRICULADOS_ULTIMOS_3',
 'EVALUADOS_ULTIMOS_3',
 'INDICE_MATEMATICAS',
 'INDICE_C_NATURALES',
 'INDICE_SOCIALES_CIUDADANAS',
 'INDICE_LECTURA_CRITICA',
 'INDICE_INGLES',
 'INDICE_TOTAL',
 'COLE_CATEGORIA',
 'rankIndiceTotal']

In [56]:
dfjoin2.select('rank','rankIndiceTotal','avg(Suma)','COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE','INDICE_TOTAL').show(10)

+----+---------------+------------------+---------------------------+-------------+------------+
|rank|rankIndiceTotal|         avg(Suma)|COLE_NOMBRE_ESTABLECIMIENTO|COLE_COD_DANE|INDICE_TOTAL|
+----+---------------+------------------+---------------------------+-------------+------------+
|   1|              1| 388.1666666666667|            LIC CAMPO DAVID| 311001065489|      0,9113|
|   2|              3| 385.6842105263158|       COL NUEVO COLOMBO...| 311001076634|       0,908|
|   3|             19|382.68518518518516|       COL NUEVA INGLATERRA| 311001031088|      0,8858|
|   4|             27|382.12820512820514|       NUEVO COLEGIO DEL...| 308001001871|      0,8797|
|   5|              7| 381.0263157894737|       COLOMBO BRITANICO...| 311001086320|      0,9005|
|   6|              6|381.00847457627117|        COL ANGLO AMERICANO| 311848000308|       0,903|
|   7|              4| 379.8048780487805|                LIC NAVARRA| 311001005176|       0,907|
|   8|             17|        